In [1]:
import pandas as pd
import numpy as np
import json
import geopandas
import geopy
import requests
from IPython.display import IFrame
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium # map rendering library
from sklearn.cluster import KMeans
import shapely.geometry
import pyproj
import csv
import numbers
import math
import tqdm

In [2]:
#Strait of the real limit
Smin_lng = 2.225684
Smin_lat = 48.885438
Smax_lng = 2.253922
Smax_lat = 48.897599

#number of cubes in two dimension
lng_count = 20
lat_count = 13

#get each step of two dimension
delta_lng = Smax_lng - Smin_lng
delta_lat = Smax_lat - Smin_lat

step_lng = delta_lng/lng_count
step_lat = delta_lat/lat_count

min_lng = Smin_lng + step_lng/2
min_lat = Smin_lat + step_lat/2
max_lng = Smax_lng - step_lng/2
max_lat = Smax_lat - step_lat/2

print(step_lng,delta_lat)

0.0014118999999999994 0.012160999999998978


In [3]:
LD_grid = pd.read_csv("grid.csv",engine='python',sep=',')
#print(LD_grid)
LD_grid['NElng'] = LD_grid['longitude'] + step_lng
LD_grid['NElat'] = LD_grid['latitude'] + step_lat
LD_grid['SWlng'] = LD_grid['longitude'] - step_lng
LD_grid['SWlat'] = LD_grid['latitude'] - step_lat
LD_grid

,grid,longitude,latitude,NElng,NElat,SWlng,SWlat
0,"0,0",2.226390,48.885906,2.227802,48.886841,2.224978,48.884970
1,"1,0",2.227802,48.885906,2.229214,48.886841,2.226390,48.884970
2,"2,0",2.229214,48.885906,2.230626,48.886841,2.227802,48.884970
3,"3,0",2.230626,48.885906,2.232038,48.886841,2.229214,48.884970
4,"4,0",2.232038,48.885906,2.233449,48.886841,2.230626,48.884970
...,...,...,...,...,...,...,...
255,"15,12",2.247568,48.897131,2.248980,48.898067,2.246157,48.896196
256,"16,12",2.248980,48.897131,2.250392,48.898067,2.247568,48.896196
257,"17,12",2.250392,48.897131,2.251804,48.898067,2.248980,48.896196
258,"18,12",2.251804,48.897131,2.253216,48.898067,2.250392,48.896196


In [4]:
# create map, LD_tweets['text']
# set location that make Hong Kong at the centre of the web page
# set zoom_start = 12 for better visualization
map = folium.Map(tiles='cartodbpositron', location=[48.89114492, 2.240388888], zoom_start=14.6)

# add markers to the map to show the districts
markers_colors = []
for lat, lon, grd in zip(LD_grid['latitude'], LD_grid['longitude'],LD_grid['grid']):
    label = folium.Popup(str(grd))
    folium.CircleMarker(
        [lat, lon],
        radius=3,
        popup=label,
        fill=True,
        fill_color='#ff6951',
        fill_opacity=0.4).add_to(map)
       
map.save('ld_grid.html')
#display(IFrame('ld_grid.html', width = 1000, height = 900))

In [5]:
# Create a function to explore the Food venure from a given coordinates
import requests
LIMIT = 50
def getNearbyVenues(names, latitudes, longitudes, NElng, NElat, SWlng, SWlat):
    
    venues_list=[]
    for name, lat, lng, nelng, nelat, swlng, swlat in zip(names, latitudes, longitudes, NElng, NElat, SWlng, SWlat):
        print(name)

        # create the API request URL
        url = 'https://api.foursquare.com/v3/places/search?ne={},{}&sw={},{}&limit={}'.format(
            nelat, 
            nelng, 
            swlat, 
            swlng, 
            LIMIT)

        headers = {
            "Accept": "application/json",
            "Authorization": "fsq37t/Q9AvUtYvgDGQ7PnLy47ErlW3mYe6I4cGh3UHnZJ0="
        }

        # make the GET request
        response = requests.request("GET", url, headers=headers).text
        resp_dict = json.loads(response)
        content = resp_dict['results']
#         print(content)

        # return only relevant information for each nearby venue
        for item in content:
            categories = item['categories']
            for cat in categories:
                venues_list.append([name,
                                    lat,
                                    lng,
                                    item['name'], 
                                    item['geocodes']['main']['latitude'],
                                    item['geocodes']['main']['longitude'],
                                    cat['name']
                                   ])
    clm = ['grid', 'gird Longitude', 'grid Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    nearby_venues = pd.DataFrame(venues_list, columns=clm)
    
    return nearby_venues

In [6]:
# Run the function for each HK districts
# Since some districts are much larger than another districts, I assume each district to be a circle and use the district area to determine the radius using
LD_grid_venues = getNearbyVenues(names = LD_grid['grid'],
                                   latitudes = LD_grid['latitude'],
                                   longitudes = LD_grid['longitude'],
                                   NElng = LD_grid['NElng'],
                                   NElat = LD_grid['NElat'],
                                   SWlng = LD_grid['SWlng'],
                                   SWlat = LD_grid['SWlat'])

0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0
10,0
11,0
12,0
13,0
14,0
15,0
16,0
17,0
18,0
19,0
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1
10,1
11,1
12,1
13,1
14,1
15,1
16,1
17,1
18,1
19,1
0,2
1,2
2,2
3,2
4,2
5,2
6,2
7,2
8,2
9,2
10,2
11,2
12,2
13,2
14,2
15,2
16,2
17,2
18,2
19,2
0,3
1,3
2,3
3,3
4,3
5,3
6,3
7,3
8,3
9,3
10,3
11,3
12,3
13,3
14,3
15,3
16,3
17,3
18,3
19,3
0,4
1,4
2,4
3,4
4,4
5,4
6,4
7,4
8,4
9,4
10,4
11,4
12,4
13,4
14,4
15,4
16,4
17,4
18,4
19,4
0,5
1,5
2,5
3,5
4,5
5,5
6,5
7,5
8,5
9,5
10,5
11,5
12,5
13,5
14,5
15,5
16,5
17,5
18,5
19,5
0,6
1,6
2,6
3,6
4,6
5,6
6,6
7,6
8,6
9,6
10,6
11,6
12,6
13,6
14,6
15,6
16,6
17,6
18,6
19,6
0,7
1,7
2,7
3,7
4,7
5,7
6,7
7,7
8,7
9,7
10,7
11,7
12,7
13,7
14,7
15,7
16,7
17,7
18,7
19,7
0,8
1,8
2,8
3,8
4,8
5,8
6,8
7,8
8,8
9,8
10,8
11,8
12,8
13,8
14,8
15,8
16,8
17,8
18,8
19,8
0,9
1,9
2,9
3,9
4,9
5,9
6,9
7,9
8,9
9,9
10,9
11,9
12,9
13,9
14,9
15,9
16,9
17,9
18,9
19,9
0,10
1,10
2,10
3,10
4,10
5,10
6,10
7,10
8,10
9,10
10,10
11,10
12,10
13,10
14,10
15,10
16,10
17,10
18

In [7]:
print(LD_grid_venues.groupby('grid').count())

      gird Longitude  grid Longitude  Venue  Venue Latitude  Venue Longitude  \
grid                                                                           
0,0                4               4      4               4                4   
0,10               6               6      6               6                6   
0,11               7               7      7               7                7   
0,12               2               2      2               2                2   
0,4                1               1      1               1                1   
...              ...             ...    ...             ...              ...   
9,5                8               8      8               8                8   
9,6               10              10     10              10               10   
9,7               13              13     13              13               13   
9,8                9               9      9               9                9   
9,9                3               3    

In [8]:
LD_grid_venues

,grid,gird Longitude,grid Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"0,0",48.885906,2.226390,Pizza Naim,48.885753,2.227217,Fast Food Restaurant
1,"0,0",48.885906,2.226390,Pizza Naim,48.885753,2.227217,Halal Restaurant
2,"0,0",48.885906,2.226390,Le Relais des Bergères,48.885372,2.225901,French Restaurant
3,"0,0",48.885906,2.226390,Rond-point des Bergères,48.885101,2.227167,Plaza
4,"1,0",48.885906,2.227802,Square president wilson,48.885685,2.228045,Park
...,...,...,...,...,...,...,...
829,"16,12",48.897131,2.248980,Grill d'Istambul,48.897184,2.247774,Brasserie
830,"16,12",48.897131,2.248980,Grill d'Istambul,48.897184,2.247774,Kebab Restaurant
831,"16,12",48.897131,2.248980,Poissonnerie la Vague (Poissonnerie La Vague),48.897357,2.247965,Fish Market
832,"18,12",48.897131,2.251804,Le Timgad,48.896628,2.253002,Hotel


In [11]:
# one hot encoding
LD_grid_venues_onehot = pd.get_dummies(LD_grid_venues[['Venue Category']], prefix="", prefix_sep="")

# add District column back to dataframe
LD_grid_venues_onehot['grid'] = LD_grid_venues['grid'] 

# move District column to the first column
fixed_columns = [LD_grid_venues_onehot.columns[-1]] + list(LD_grid_venues_onehot.columns[:-1])
LD_grid_venues_onehot = LD_grid_venues_onehot[fixed_columns]

LD_grid_venues_onehot.head()

,grid,American Restaurant,Amusement Park,Bakery,Bank,Banking and Finance,Bar,Basketball Court,Beer Bar,Beer Store,...,Sporting Goods Retail,Stadium,Steakhouse,Sushi Restaurant,Swimming Pool,Thai Restaurant,Turkish Restaurant,Vietnamese Restaurant,Wine Store,Women's Store
0,"0,0",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"0,0",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"0,0",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"0,0",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"1,0",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
LD_grid_grouped = LD_grid_venues_onehot.groupby('grid').mean().reset_index()
LD_grid_grouped

,grid,American Restaurant,Amusement Park,Bakery,Bank,Banking and Finance,Bar,Basketball Court,Beer Bar,Beer Store,...,Sporting Goods Retail,Stadium,Steakhouse,Sushi Restaurant,Swimming Pool,Thai Restaurant,Turkish Restaurant,Vietnamese Restaurant,Wine Store,Women's Store
0,"0,0",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
1,"0,10",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
2,"0,11",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
3,"0,12",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
4,"0,4",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,"9,5",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125
188,"9,6",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100
189,"9,7",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
190,"9,8",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000


In [13]:
# Find out the top 10 restaurant type in the district

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['grid']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most poi Type'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most poi Type'.format(ind+1))

# create a new dataframe
LD_grid_dish_sorted = pd.DataFrame(columns=columns)
LD_grid_dish_sorted['grid'] = LD_grid_grouped['grid']

for ind in np.arange(LD_grid_grouped.shape[0]):
    LD_grid_dish_sorted.iloc[ind, 1:] = return_most_common_venues(LD_grid_grouped.iloc[ind, :], num_top_venues)

LD_grid_dish_sorted

,grid,1st Most poi Type,2nd Most poi Type,3rd Most poi Type,4th Most poi Type,5th Most poi Type,6th Most poi Type,7th Most poi Type,8th Most poi Type,9th Most poi Type,10th Most poi Type
0,"0,0",Halal Restaurant,Fast Food Restaurant,French Restaurant,Plaza,Mexican Restaurant,Pedestrian Plaza,Pastry Shop,Park,Movie Theater,Moroccan Restaurant
1,"0,10",Restaurant,Japanese Restaurant,Burger Joint,Brasserie,Italian Restaurant,Indian Restaurant,Pizzeria,Pedestrian Plaza,Pastry Shop,Park
2,"0,11",Restaurant,Burger Joint,Japanese Restaurant,Legal Service,Financial Service,Brasserie,American Restaurant,Pedestrian Plaza,Pastry Shop,Park
3,"0,12",Financial Service,Legal Service,American Restaurant,Mexican Restaurant,Pedestrian Plaza,Pastry Shop,Park,Movie Theater,Moroccan Restaurant,Modern European Restaurant
4,"0,4",Miscellaneous Store,American Restaurant,Health and Medicine,Pizzeria,Pedestrian Plaza,Pastry Shop,Park,Movie Theater,Moroccan Restaurant,Modern European Restaurant
...,...,...,...,...,...,...,...,...,...,...,...
187,"9,5",Clothing Store,Women's Store,Sporting Goods Retail,Furniture and Home Store,Burrito Restaurant,Cosmetics Store,Swimming Pool,Sushi Restaurant,Pedestrian Plaza,Pastry Shop
188,"9,6",Women's Store,Sporting Goods Retail,Landmarks and Outdoors,Furniture and Home Store,Drugstore,Retail,Clothing Store,Sandwich Restaurant,Burrito Restaurant,Cosmetics Store
189,"9,7",Retail,Shoe Store,Miscellaneous Store,Fast Food Restaurant,Computers and Electronics Retail,Clothing Store,Salad Restaurant,Sandwich Restaurant,Food and Beverage Retail,Drugstore
190,"9,8",Shoe Store,Miscellaneous Store,Computers and Electronics Retail,Retail,Clothing Store,Fast Food Restaurant,Salad Restaurant,Food and Beverage Retail,Plaza,Lounge


In [14]:
# set number of clusters
kclusters = 12

# set the attributes used for clustering
LD_grid_grouped_clustering = LD_grid_grouped.drop('grid', 1)

# run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state = 0)
kmeans.fit(LD_grid_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

C:\Users\admin\AppData\Local\Temp\ipykernel_13260\266218820.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  LD_grid_grouped_clustering = LD_grid_grouped.drop('grid', 1)


array([ 0,  0,  0,  0, 10, 10,  0,  0,  0,  0])

In [15]:
# Merge the result into the district dataframe
# add clustering labels
LD_grid_dish_sorted['Cluster Labels'] = kmeans.labels_
#HK_district_dish_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

LD_grid_merged = LD_grid

# merge df_grouped with df_data to add latitude/longitude for each neighbourhood
LD_grid_merged = LD_grid_merged.join(LD_grid_dish_sorted.set_index('grid'), on='grid')
LD_grid_merged['Cluster Labels']=[int(val) if isinstance(val,numbers.Number) & (math.isnan(val)==False) else 0 for val in LD_grid_merged['Cluster Labels']]
LD_grid_merged

,grid,longitude,latitude,NElng,NElat,SWlng,SWlat,1st Most poi Type,2nd Most poi Type,3rd Most poi Type,4th Most poi Type,5th Most poi Type,6th Most poi Type,7th Most poi Type,8th Most poi Type,9th Most poi Type,10th Most poi Type,Cluster Labels
0,"0,0",2.226390,48.885906,2.227802,48.886841,2.224978,48.884970,Halal Restaurant,Fast Food Restaurant,French Restaurant,Plaza,Mexican Restaurant,Pedestrian Plaza,Pastry Shop,Park,Movie Theater,Moroccan Restaurant,0
1,"1,0",2.227802,48.885906,2.229214,48.886841,2.226390,48.884970,Halal Restaurant,Fast Food Restaurant,Park,Plaza,Bank,Italian Restaurant,Modern European Restaurant,Pizzeria,Pedestrian Plaza,Pastry Shop,0
2,"2,0",2.229214,48.885906,2.230626,48.886841,2.227802,48.884970,Park,American Restaurant,Health and Medicine,Pizzeria,Pedestrian Plaza,Pastry Shop,Movie Theater,Moroccan Restaurant,Modern European Restaurant,Miscellaneous Store,5
3,"3,0",2.230626,48.885906,2.232038,48.886841,2.229214,48.884970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,"4,0",2.232038,48.885906,2.233449,48.886841,2.230626,48.884970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,"15,12",2.247568,48.897131,2.248980,48.898067,2.246157,48.896196,Fast Food Restaurant,Creperie,Brasserie,French Restaurant,Food and Beverage Retail,Fish Market,Restaurant,Convenience Store,Café,Burger Joint,0
256,"16,12",2.248980,48.897131,2.250392,48.898067,2.247568,48.896196,Restaurant,Kebab Restaurant,Fast Food Restaurant,Beer Bar,Fish Market,Brasserie,Café,Food and Beverage Retail,Moroccan Restaurant,Pizzeria,0
257,"17,12",2.250392,48.897131,2.251804,48.898067,2.248980,48.896196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
258,"18,12",2.251804,48.897131,2.253216,48.898067,2.250392,48.896196,Hotel,Health and Medicine,Pizzeria,Pedestrian Plaza,Pastry Shop,Park,Movie Theater,Moroccan Restaurant,Modern European Restaurant,Miscellaneous Store,6


In [16]:
# create map
map_cluster = folium.Map(tiles='cartodbpositron', location=[48.89114492, 2.240388888], zoom_start=14.6)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(LD_grid_merged['latitude'], LD_grid_merged['longitude'], LD_grid_merged['grid'], LD_grid_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=3,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.5).add_to(map_cluster)
       
map_cluster.save('ld_district_cluster.html')
display(IFrame('ld_district_cluster.html', width = 1500, height = 1200))

In [ ]:
LD_grid_merged[LD_grid_merged['Cluster Labels'] == 3]

In [18]:
LD_grid_venues.to_csv('data.csv', index=False, encoding='utf-8')

In [1]:
import requests

url = "https://api.foursquare.com/v2/venues/search
  ?client_id=3ZK4LUBW25RFO1X1FU5KNVRTUJQTZI1TR2GEH43OHI0FYTTQ
  &client_secret=OOOPPVM3PRPG4Q2PYEXYUM0HHQHDKSYPSY4FYONDSHH5FOSR
  &ll=40.7,-74
  &query=sushi
  &v=20220128"

headers = {"Accept": "application/json"}

response = requests.request("GET", url, headers=headers)

print(response.text)

SyntaxError: EOL while scanning string literal (3877533039.py, line 3)